In [ ]:
# Install OCR Library
pip install pytesseract opencv-python easyocr

In [ ]:
import cv2
import pytesseract
from ultralytics import YOLO

# Set Tesseract executable path (Windows)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load YOLO model
model = YOLO('runs/train/product_detect/weights/best.pt')

# Load input image
image = cv2.imread('test_image.jpg')

# Run object detection
results = model(image)

# Run OCR on each detected bounding box region
for box in results[0].boxes.xyxy:
    x1, y1, x2, y2 = map(int, box)  # Extract coordinates
    roi = image[y1:y2, x1:x2]       # Region of interest (ROI)

    # Apply OCR
    text = pytesseract.image_to_string(roi, lang='eng')
    print("Detected Text:", text)

    # (Optional) Extract product name using keyword matching
    if "제품명" in text:
        print(f"Detected Product Name: {text.strip()}")

# Visualize detection result
cv2.imshow('Detected', image)
cv2.waitKey(0)

In [ ]:
!pip install google-cloud-vision

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.cloud import vision
from google.cloud.vision_v1 import types
import re
import os

# -----------------------------
# NOTE: Secret credentials removed.
# Set your Google Vision credentials before running, e.g.:
# export GOOGLE_APPLICATION_CREDENTIALS="path/to/your_credentials.json"
# or in Colab:
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/your_credentials.json"
# -----------------------------

# 1. Input image path
image_path = '/content/test.jpg'

# 2. Read image
image = cv2.imread(image_path)
if image is None:
    print("Cannot read the image. Please check the file path.")
    exit()

# 3. Preprocessing
# (1) Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# (2) Adaptive thresholding (binarization)
adaptive_thresh = cv2.adaptiveThreshold(
    gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
)

# (3) Sharpening
kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
sharpened = cv2.filter2D(adaptive_thresh, -1, kernel)

# (4) Morphological operation
kernel_morph = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
morphed = cv2.morphologyEx(sharpened, cv2.MORPH_CLOSE, kernel_morph)

# 4. Run Google Vision OCR
client = vision.ImageAnnotatorClient()

# Read image as binary for Vision API
with open(image_path, "rb") as image_file:
    content = image_file.read()

image_vision = types.Image(content=content)
response = client.text_detection(image=image_vision)

# OCR result
texts = response.text_annotations

# 5. Keyword-based extraction (tag / nutri / date)
if texts:
    print("OCR Results:")
    for text in texts:
        detected_text = text.description

        # tag class (example regex)
        match_tag = re.search(r'명\s+(\S+)', detected_text)
        if match_tag:
            print(f"Extracted text from TAG region: {match_tag.group(1)}")

        # nutri class (example regex)
        match_nutri = re.search(r'영양정보(.*?)kcal', detected_text)
        if match_nutri:
            print(f"Extracted text from NUTRI region: {match_nutri.group(1)}")

        # date class (example regex)
        match_date = re.search(r'(24|25)(.*)', detected_text)
        if match_date:
            print(f"Extracted text from DATE region: {match_date.group(2)}")
else:
    print("No text detected.")

# Show original and preprocessed images
print("Original image:")
cv2_imshow(image)

print("Preprocessed image:")
cv2_imshow(morphed)